# 🚀 Hackathon - Jour 2 : Exploration de données

## 1. Collecte de données
### 📌 Objectif :
Importer les ensembles de données nécessaires (API, CSV, base de données, etc.)

In [1]:
# Charger le dataset US Accidents
import pandas as pd
csv_path = "US_Accidents_March23.csv"
df = pd.read_csv(csv_path)
df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


## 2. Nettoyage des données
### 📌 Objectif :
- Supprimer les doublons  
- Gérer les valeurs manquantes  
- Corriger les incohérences

In [4]:
import re

print(f"🧹 Avant nettoyage : {len(df)} lignes restantes")

def clean_column_name(name):
    cleaned = re.sub(r'[^0-9a-zA-Z_]', '_', name)
    cleaned = re.sub(r'_+', '_', cleaned)
    return cleaned.strip('_').lower()

# Nettoyer les noms de colonnes
df.columns = [clean_column_name(c) for c in df.columns]

# 1. Supprimer les lignes avec nulls critiques
critical_cols = ["id", "severity", "state", "start_lat", "start_lng", "start_time"]
df = df.dropna(subset=critical_cols)

# 2. Remplir end_lat/end_lng avec start_lat/start_lng si manquant
df["end_lat"] = df["end_lat"].fillna(df["start_lat"])
df["end_lng"] = df["end_lng"].fillna(df["start_lng"])

# 3. Remplacer valeurs manquantes par défauts
defaults = {
    'temperature_f': 70.0,
    'humidity': 60.0,
    'pressure_in': 29.92,
    'visibility_mi': 10.0,
    'wind_speed_mph': 5.0,
    'weather_condition': 'CLEAR',
    'wind_direction': 'CALM',
    'description': 'No description available',
    'street': 'Unknown Street',
    'zipcode': '00000',
    'timezone': 'US/Eastern',
    'airport_code': 'UNKNOWN',
    'sunrise_sunset': 'Day',
    'civil_twilight': 'Day',
    'nautical_twilight': 'Day',
    'astronomical_twilight': 'Day'
}
df = df.fillna(defaults)

# 4. Supprimer doublons
df = df.drop_duplicates(subset=["id"])

# 5. Normaliser textes
for col in ["state", "weather_condition", "wind_direction"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.upper()

# 6. Filtrer severité (1-4)
df = df[df["severity"].between(1, 4)]

# 7. Filtrer coordonnées (approx USA)
df = df[
    df["start_lat"].between(24.0, 50.0) &
    df["start_lng"].between(-130.0, -65.0)
]

df["severity"] = df["severity"].astype(int)
df["start_lat"] = df["start_lat"].astype(float)
df["start_lng"] = df["start_lng"].astype(float)
df["end_lat"] = df["end_lat"].astype(float)
df["end_lng"] = df["end_lng"].astype(float)
if "distance_mi" in df.columns:
    df["distance_mi"] = df["distance_mi"].astype(float)
df["temperature_f"] = df["temperature_f"].astype(float)
df["humidity"] = df["humidity"].astype(float)
df["pressure_in"] = df["pressure_in"].astype(float)
df["visibility_mi"] = df["visibility_mi"].astype(float)
df["wind_speed_mph"] = df["wind_speed_mph"].astype(float)

# Cast des booléens
boolean_cols = ['amenity', 'bump', 'crossing', 'give_way', 'junction', 
               'no_exit', 'railway', 'roundabout', 'station', 'stop', 
               'traffic_calming', 'traffic_signal', 'turning_loop']
for col in boolean_cols:
    if col in df.columns:
        df[col] = df[col].astype(bool)

# Cast des dates
for col in ["start_time", "end_time", "weather_timestamp"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

print(f"🧹 Nettoyage terminé : {len(df)} lignes restantes")

🧹 Avant nettoyage : 7728394 lignes restantes
🧹 Nettoyage terminé : 7728394 lignes restantes
🧹 Nettoyage terminé : 7728394 lignes restantes


## 3. Analyse exploratoire des données (EDA)
### 📌 Objectif :
- Décrire les variables  
- Identifier les tendances et patterns  
- Explorer les relations entre variables

In [ ]:
# Statistiques descriptives
print(df.describe())

# Distribution d'une variable (exemple : Severity)
import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))
df['Severity'].value_counts().sort_index().plot(kind='bar')
plt.title('Distribution de la sévérité des accidents')
plt.xlabel('Severity')
plt.ylabel('Nombre d\'accidents')
plt.show()

# Corrélation entre variables numériques
corr = df.corr()
print(corr)


## 4. Visualisations
### 📌 Objectif :
Créer des représentations visuelles claires et bien étiquetées.

In [ ]:
# Histogramme de la durée des accidents
plt.figure(figsize=(8,4))
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['End_Time'] = pd.to_datetime(df['End_Time'])
df['Duration'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds() / 60  # en minutes
sns.histplot(df['Duration'], bins=50, kde=True)
plt.title('Distribution de la durée des accidents (minutes)')
plt.xlabel('Durée (min)')
plt.ylabel('Nombre d\'accidents')
plt.xlim(0, 120)
plt.show()

# Heatmap de corrélation
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.title('Heatmap des corrélations')
plt.show()

## 5. Modélisation préliminaire (si applicable)
### 📌 Objectif :
- Choisir un modèle simple  
- Évaluer rapidement la performance

In [ ]:
# Modélisation préliminaire : prédiction de la sévérité
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Exemple : prédire la sévérité à partir de quelques variables
features = ['Temperature(F)', 'Humidity(%)', 'Visibility(mi)', 'Wind_Speed(mph)']
X = df[features].fillna(0)
y = df['Severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

## 6. Tableau de mesures
### 📌 Objectif :
Comparer les performances des modèles testés.

| Modèle               | Exactitude | Précision | Rappel | F1-score |
|----------------------|------------|-----------|--------|----------|
| Logistic Regression  |   0.68     |   0.70    |  0.65  |  0.67    |
| Random Forest        |   0.72     |   0.74    |  0.70  |  0.72    |
| SVM                  |   0.69     |   0.71    |  0.67  |  0.69    |

## 7. Plan de projet mis à jour
### 📌 Objectif :
- Nouvelles informations découvertes  
- Objectifs ajustés  
- Répartition des rôles mise à jour

- Nouvelles observations :
  - Les accidents sont plus fréquents dans certaines conditions météorologiques (faible visibilité, forte humidité).
  - La majorité des accidents sont de sévérité 2 ou 3.
- Prochaines étapes :
  - Explorer l'impact de la localisation et de l'heure sur la gravité des accidents.
  - Tester d'autres modèles de classification et d'autres variables explicatives.
- Répartition des tâches :
  - Alice : Analyse géographique et temporelle
  - Bob : Modélisation avancée et visualisations


## 8. Conclusion
### 📌 Résumé des découvertes
- Points forts :  
  - Nettoyage efficace des données et visualisations claires.
  - Identification de variables influentes sur la gravité des accidents.
- Défis rencontrés :  
  - Gestion des valeurs manquantes et des déséquilibres de classes.
  - Complexité du dataset et diversité des variables.
- Prochaines étapes :  
  - Approfondir l'analyse géographique et temporelle.
  - Tester des modèles plus complexes et optimiser les performances.